<a href="https://colab.research.google.com/github/alokchoudharyguliya/FineTuning/blob/main/FineTuningUsingLoRA_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate datasets peft bitsandbytes safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.5 MB/s eta 0:00:00


In [2]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
# from datasets import load_dataset
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# model_name = "openai-community/gpt2"  # replace with the model you want to adapt
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token

# # load model (FP32 or FP16)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
# model = prepare_model_for_kbit_training(model)  # safe if you plan to use k-bit (but okay otherwise)

# # LoRA config
# lora_config = LoraConfig(
#     r=8,                     # LoRA rank
#     lora_alpha=16,           # scaling
#     target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "fc1", "fc2"],  # adapt these names to model
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# model = get_peft_model(model, lora_config)

# # dataset example: some small text dataset
# ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")
# def tokenize(ex):
#     return tokenizer(ex["text"], truncation=True, padding="max_length", max_length=512)
# ds = ds.map(tokenize, batched=True)
# ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

# training_args = TrainingArguments(
#     output_dir="./lora-gpt2",
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=8,
#     num_train_epochs=3,
#     learning_rate=2e-4,
#     fp16=True,
#     logging_steps=20,
#     optim="adamw_torch",
#     save_total_limit=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=ds,
#     tokenizer=tokenizer,
# )

# trainer.train()
# # save just LoRA adapters
# # model.save_pretrained("./lora-gpt2-adapter")


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model = prepare_model_for_kbit_training(model)

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # For GPT-2, use these instead of q_proj, v_proj etc.
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# Load dataset
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")

def tokenize(examples):
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

ds = ds.map(tokenize, batched=True)
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Training args
training_args = TrainingArguments(
    output_dir="./lora-gpt2",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    optim="adamw_torch",
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    tokenizer=tokenizer,
)

trainer.train()

# Save just LoRA adapters
model.save_pretrained("./lora-gpt2-adapter")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: waynerooney0089 (waynerooney0089-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
20,6.762100
40,2.054300
60,1.145200


TrainOutput(global_step=69, training_loss=3.0077016388160596, metrics={'train_runtime': 215.257, 'train_samples_per_second': 5.115, 'train_steps_per_second': 0.321, 'total_flos': 290425576882176.0, 'train_loss': 3.0077016388160596, 'epoch': 3.0})

In [3]:
# requirements:
# pip install transformers accelerate datasets peft bitsandbytes einops safetensors

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from accelerate import init_empty_weights, infer_auto_device_map

MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"  # example; replace as needed

# Create bitsandbytes config for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,   # double quantization
    bnb_4bit_quant_type="nf4",        # NF4 quant format
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
# load model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",     # or use infer_auto_device_map for manual
    trust_remote_code=True,
)

# Prepare for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA config (common for QLoRA)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # check module names
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# Load dataset & tokenize (as before)
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:0.5%]")
def tokenize(ex):
    return tokenizer(ex["text"], truncation=True, padding="max_length", max_length=512)
ds = ds.map(tokenize, batched=True)
ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Use accelerate-compatible Trainer or a custom training loop
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./q-lora-llama",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_total_limit=1,
    optim="adamw_torch",
    report_to=["none"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    tokenizer=tokenizer,
)

trainer.train()
model.save_pretrained("./q-lora-llama-adapter")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-68f693f6-7d1923c04043dfcb29d2e424;4a11bdf1-0cdd-4279-8a9f-7753e272d7c9)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.

In [4]:
!zip -r lora-gpt2.zip lora-gpt2/
from google.colab import files
files.download('lora-gpt2.zip')

  adding: lora-gpt2/ (stored 0%)
  adding: lora-gpt2/runs/ (stored 0%)
  adding: lora-gpt2/runs/Oct20_19-52-58_fe244db27506/ (stored 0%)
  adding: lora-gpt2/runs/Oct20_19-52-58_fe244db27506/events.out.tfevents.1760989982.fe244db27506.281.0 (deflated 60%)
  adding: lora-gpt2/checkpoint-69/ (stored 0%)
  adding: lora-gpt2/checkpoint-69/scaler.pt (deflated 64%)
  adding: lora-gpt2/checkpoint-69/training_args.bin (deflated 53%)
  adding: lora-gpt2/checkpoint-69/vocab.json (deflated 59%)
  adding: lora-gpt2/checkpoint-69/tokenizer.json (deflated 82%)
  adding: lora-gpt2/checkpoint-69/trainer_state.json (deflated 59%)
  adding: lora-gpt2/checkpoint-69/adapter_model.safetensors (deflated 8%)
  adding: lora-gpt2/checkpoint-69/rng_state.pth (deflated 26%)
  adding: lora-gpt2/checkpoint-69/optimizer.pt (deflated 8%)
  adding: lora-gpt2/checkpoint-69/special_tokens_map.json (deflated 60%)
  adding: lora-gpt2/checkpoint-69/tokenizer_config.json (deflated 54%)
  adding: lora-gpt2/checkpoint-69/READ

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!zip -r lora-wanddb.zip wandb/
from google.colab import files
files.download('lora-wanddb.zip')

  adding: wandb/ (stored 0%)
  adding: wandb/debug.log (deflated 69%)
  adding: wandb/debug-internal.log (deflated 69%)
  adding: wandb/run-20251020_195527-5mf1kiwm/ (stored 0%)
  adding: wandb/run-20251020_195527-5mf1kiwm/tmp/ (stored 0%)
  adding: wandb/run-20251020_195527-5mf1kiwm/tmp/code/ (stored 0%)
  adding: wandb/run-20251020_195527-5mf1kiwm/run-5mf1kiwm.wandb (deflated 85%)
  adding: wandb/run-20251020_195527-5mf1kiwm/files/ (stored 0%)
  adding: wandb/run-20251020_195527-5mf1kiwm/files/wandb-metadata.json (deflated 40%)
  adding: wandb/run-20251020_195527-5mf1kiwm/files/requirements.txt (deflated 56%)
  adding: wandb/run-20251020_195527-5mf1kiwm/files/output.log (deflated 69%)
  adding: wandb/run-20251020_195527-5mf1kiwm/logs/ (stored 0%)
  adding: wandb/run-20251020_195527-5mf1kiwm/logs/debug-core.log (deflated 56%)
  adding: wandb/run-20251020_195527-5mf1kiwm/logs/debug.log (deflated 69%)
  adding: wandb/run-20251020_195527-5mf1kiwm/logs/debug-internal.log (deflated 69%)
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>